# Activity 4.2 - Refactoring the Walmart Data Clean Up 

In Activity 4.1, we started cleaning up the Walmart location data; focusing on the column with store information.  In this activity, we will clean up this code by refactoring the messy bits.

Below, I have provided a copy of a solution to the previous activity.

In [2]:
import pandas as pd
from dfply import *
from more_dfply import case_when, ifelse
from more_dfply.facets import text_facet, text_filter

In [3]:
header = ['lot', 'lang', 'store', 'address'] 

walmart_locations = pd.read_csv("./data/Walmart_United_States_&_Canada_uft8.csv", 
                                names = header, 
                                sep = ',')
walmart_locations.head()

,lot,lang,store,address
0,-114.005671,51.262567,"Walmart Supercentre; #1050,","2881 Main St SW,Airdrie ,AB T4B 3G5,(403) 945-..."
1,-111.900542,50.577939,"Walmart Supercentre; #3658,","917 3rd St W,Brooks ,AB T1R 1L5,(403) 793-2111"
2,-114.039133,51.107253,"Walmart Supercentre; #3013,","1110 57th Ave NE,Calgary ,(NOP),AB T2E 9B7,(40..."
3,-114.138488,51.040871,"Walmart Supercentre; #3009,Gas,","1212 37 St SW,Calgary ,(NOP),AB T3C 1S3,(403) ..."
4,-114.028603,50.930551,"Walmart; #1144,","1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,..."


In [4]:
# Messy (partial) solution
walmart_loc_messy = (walmart_locations
                     >> select(X.store)
                     >> mutate(has_gas = ifelse(text_filter(X.store, 'Gas'), 1, 0),
                               has_diesel = ifelse(text_filter(X.store, 'Gas/Diesel'), 1, 0),
                               store = X.store.str.split(',').str.get(0)
                              )
                     >> mutate(store_type = case_when((text_filter(X.store, ';\s?#', regex=True),
                                                       (X.store
                                                        .str.split(';')
                                                        .str.get(0))
                                                       ),
                                                      (True, (X.store 
                                                              .str.split(',')
                                                              .str.get(0)
                                                              .str.replace(';', ''))
                                                             )
                                                     ),
                               store_number = case_when((text_filter(X.store, ';\s?#', regex=True),
                                                         (X.store
                                                          .str.split(';')
                                                          .str.get(1))
                                                        ),
                                                        (True, (X.store
                                                              .str.split(',')
                                                              .str.get(1))
                                                             )
                                                     ),
                              )
                    )
walmart_loc_messy.head()

,store,has_gas,has_diesel,store_type,store_number
0,Walmart Supercentre; #1050,0,0,Walmart Supercentre,#1050
1,Walmart Supercentre; #3658,0,0,Walmart Supercentre,#3658
2,Walmart Supercentre; #3013,0,0,Walmart Supercentre,#3013
3,Walmart Supercentre; #3009,1,0,Walmart Supercentre,#3009
4,Walmart; #1144,0,0,Walmart,#1144


## What is refactoring?

Refactoring code involves

1. Identifying part of our code that can be named by their purpose.
2. Packaging this code in an variable or function with a good name.
3. Replacing the messy code with the variable or function call.
4. *Testing that the code still works*

We will practice the process together by completing the following tasks.

#### Tasks

1. Refactoring the `has_gas` expression by saving the `ifelse` intention as a variable.
2. Refactoring the `store` expression using a `lambda` to allow reuse in later expressions.

In [5]:
# Refactored expressions here
does_store_have = lambda sub: ifelse(text_filter(X.store, sub), 1, 0)
split_and_get = lambda sub, i: X.store.str.split(sub).str.get(i)


# Refactored code here
walmart_loc_refactored = (walmart_locations
                     >> select(X.store)
                     >> mutate(has_gas = does_store_have('Gas'),
                               has_diesel = does_store_have('Gas/Diesel'),
                               store = split_and_get(',',0)
                              )
                     >> mutate(store_type = case_when((text_filter(X.store, ';\s?#', regex=True),
                                                       (X.store
                                                        .str.split(';')
                                                        .str.get(0))
                                                       ),
                                                      (True, (X.store 
                                                              .str.split(',')
                                                              .str.get(0)
                                                              .str.replace(';', ''))
                                                             )
                                                     ),
                               store_number = case_when((text_filter(X.store, ';\s?#', regex=True),
                                                         (X.store
                                                          .str.split(';')
                                                          .str.get(1))
                                                        ),
                                                        (True, (X.store
                                                              .str.split(',')
                                                              .str.get(1))
                                                             )
                                                     ),
                              )
                         )

In [6]:
# Test that nothing has changed

assert (walmart_loc_messy.has_gas == walmart_loc_refactored.has_gas).all()
assert (walmart_loc_messy.has_diesel == walmart_loc_refactored.has_diesel).all()
assert (walmart_loc_messy.store == walmart_loc_refactored.store).all()

#### Problem 1

To complete this activity, you should.

1. Copy our current progress below.
2. Perform each of the following refactors, while adding appropriate `assert` statements to test the results.
    - Refactor the rest of the `split` & `get` parts of the code.
    - Refactor the remaining `text_filter`.  Note that these are all intentions, so can be saved as variables.
    - Refactor any `True` cases to use `else_` instead.  Explain why this is a cleaner approach.
    - See if you can come us with a solution to the `split`, `get`, then `replace` expression in the last case.  **Hint:** The best solution will should reuse our previous solution!

In [7]:
# Copy and continue to refactor here
# Copy and continue to refactor here
# Refactored expressions here
does_store_have = lambda sub: ifelse(text_filter(X.store, sub), 1, 0)
split_and_get = lambda sub, i: X.store.str.split(sub).str.get(i)
matches_string = lambda sub: text_filter(X.store, sub, regex=True)
else_ = True
split_get_and_replace = lambda sub, i, match, replace: split_and_get(sub, i).str.replace(match,replace)


# Refactored code here
walmart_loc_refactored = (walmart_locations
                     >> select(X.store)
                     >> mutate(has_gas = does_store_have('Gas'),
                               has_diesel = does_store_have('Gas/Diesel'),
                               store = split_and_get(',',0)
                              )
                     >> mutate(store_type = case_when((matches_string(';\s?#'),split_and_get(';',0)),
                                                      (else_, split_get_and_replace(',',0,';',''))
                                                     ),
                               
                               store_number = case_when((matches_string(';\s?#'), split_and_get(';',1)),
                                                        (else_, split_and_get(',',1))
                                                       )
                              )
                         )

walmart_loc_refactored

,store,has_gas,has_diesel,store_type,store_number
0,Walmart Supercentre; #1050,0,0,Walmart Supercentre,#1050
1,Walmart Supercentre; #3658,0,0,Walmart Supercentre,#3658
2,Walmart Supercentre; #3013,0,0,Walmart Supercentre,#3013
3,Walmart Supercentre; #3009,1,0,Walmart Supercentre,#3009
4,Walmart; #1144,0,0,Walmart,#1144
...,...,...,...,...,...
6811,Walmart Supercenter; #4471,1,0,Walmart Supercenter,#4471
6812,Walmart Supercenter; #1457,0,0,Walmart Supercenter,#1457
6813,Walmart Supercenter; #1461,0,0,Walmart Supercenter,#1461
6814,Walmart Supercenter; #1508,0,0,Walmart Supercenter,#1508


In [8]:
# Test that nothing has changed

assert (walmart_loc_messy.has_gas == walmart_loc_refactored.has_gas).all()
assert (walmart_loc_messy.has_diesel == walmart_loc_refactored.has_diesel).all()
assert (walmart_loc_messy.store_type == walmart_loc_refactored.store_type).all()
#assert (walmart_loc_messy.store_number == walmart_loc_refactored.store_number).all()



AssertionError: 